In [ ]:
import pandas as pd


df = pd.read_csv("Iris.csv").drop(["Id"], axis=1)

class_names = df["Species"].unique().tolist()
col_labels = ["Size", "Features", "Classes"] + class_names + ["Undefined objects"]

df = df.map(lambda element: pd.NA if element in ['', ' ', None] else element)
undefined_data = round((df.isna().any(axis=1).sum() / df.size) * 100, 3)
objects_in_classes = df["Species"].value_counts().to_list()
cells = [[df["Species"].size, 4, 3] + objects_in_classes + [undefined_data]]

dataset_info = pd.DataFrame(cells, columns=col_labels)
print(dataset_info)

   Size  Features  Classes  Iris-setosa  Iris-versicolor  Iris-virginica  \
0   150         4        3           50               50              50   

   Undefined objects  
0                0.0  
